# Deep neural networks (basics)

In [1]:
import tensorflow as tf

## ReLU

Build a two layer DNN with ReLU activation function

<img src="image/relu-network.png" width="400"/>

<img src="image/network.png" width="500" />

In [26]:
# Input (4 features, 3 data points)
features = tf.Variable([[1.0, 2.0, 3.0, 4.0], [-1.0, -2.0, -3.0, -4.0], [11.0, 12.0, 13.0, 14.0]])

# # Weights and biases

# 4 x 3 weights for layer 1
hidden_layer_weights = [
    [0.1, 0.2, 0.4],
    [0.4, 0.6, 0.6],
    [0.5, 0.9, 0.1],
    [0.8, 0.2, 0.8]]
# 3 x 2 weights for layer 2
out_weights = [
    [0.1, 0.6],
    [0.2, 0.1],
    [0.7, 0.9]]
weights = [
    tf.Variable(hidden_layer_weights),
    tf.Variable(out_weights)]

biases = [
    tf.Variable(tf.zeros(3)),
    tf.Variable(tf.zeros(2))]

output = None

In [23]:
print('Input shape:', features.shape)
print('Layer 1 weights:', weights[0].shape)
print('Layer 1 biases:', biases[0].shape)
print('Layer 2 weights:', weights[1].shape)
print('Layer 2 biases:', biases[1].shape)

Input shape: (3, 4)
Layer 1 weights: (4, 3)
Layer 1 biases: (3,)
Layer 2 weights: (3, 2)
Layer 2 biases: (2,)


In [24]:
# model
hidden_layer_in = tf.add(tf.matmul(features, weights[0]), biases[0])
hidden_layer_out = tf.nn.relu(hidden_layer_in)
logits = tf.add(tf.matmul(hidden_layer_out, weights[1]), biases[1])

In [27]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output = sess.run(logits)
print('Output:', output.shape)
print('3 points, each with 2 labels')
print(output)

Output: (3, 2)
3 points, each with 2 labels
[[ 5.11      8.440001]
 [ 0.        0.      ]
 [24.010002 38.239998]]


## DNN Example

MNIST Dataset

* input_layer: 784 features
* hidden_layer: 256 features
* output_layer: 10 classes

In [28]:
# import data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

/Users/rumman/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [31]:
# parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 128  # Decrease batch size if you don't have enough memory
display_step = 1

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)
n_hidden_layer = 256 # layer number of features

In [32]:
# Store layers weight & bias
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [33]:
# tf Graph input
x = tf.placeholder("float", [None, 28, 28, 1])
y = tf.placeholder("float", [None, n_classes])

x_flat = tf.reshape(x, [-1, n_input])

In [35]:
# Hidden layer with RELU activation
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']),\
    biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)
# Output layer with linear activation
logits = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])

In [36]:
# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

In [37]:
# Launch the graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Training cycle
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
        # Display logs per epoch step
        if epoch % display_step == 0:
            c = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(c))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    # Decrease test_size if you don't have enough memory
    test_size = 256
    print("Accuracy:", accuracy.eval({x: mnist.test.images[:test_size], y: mnist.test.labels[:test_size]}))

Epoch: 0001 cost= 42.848636627
Epoch: 0002 cost= 31.394573212
Epoch: 0003 cost= 21.191545486
Epoch: 0004 cost= 15.748582840
Epoch: 0005 cost= 15.114926338
Epoch: 0006 cost= 11.691316605
Epoch: 0007 cost= 13.181984901
Epoch: 0008 cost= 11.000345230
Epoch: 0009 cost= 11.741348267
Epoch: 0010 cost= 11.759691238
Epoch: 0011 cost= 13.137220383
Epoch: 0012 cost= 5.777841568
Epoch: 0013 cost= 7.330555916
Epoch: 0014 cost= 11.192827225
Epoch: 0015 cost= 8.395056725
Epoch: 0016 cost= 8.178954124
Epoch: 0017 cost= 8.586244583
Epoch: 0018 cost= 12.249856949
Epoch: 0019 cost= 8.305903435
Epoch: 0020 cost= 10.131723404
Optimization Finished!
Accuracy: 0.8125


## Saving and restore

### Saving variables

In [41]:
# The file path to save the data
save_file = './model.ckpt'

# Two Tensor Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))

# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()

with tf.Session() as sess:
    # Initialize all the Variables
    sess.run(tf.global_variables_initializer())

    # Show the values of weights and bias
    print('Weights:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

    # Save the model
    saver.save(sess, save_file)

Weights:
[[-0.3668472   0.6744311   0.8026446 ]
 [ 1.3002928   0.2349743   0.03514677]]
Bias:
[ 0.8797194  -0.09999978 -0.31319168]


### Loading variables

In [42]:
# Remove the previous weights and bias
tf.reset_default_graph()

# Two Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]))
bias = tf.Variable(tf.truncated_normal([3]))

# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()

with tf.Session() as sess:
    # Load the weights and bias
    saver.restore(sess, save_file)

    # Show the values of weights and bias
    print('Weight:')
    print(sess.run(weights))
    print('Bias:')
    print(sess.run(bias))

INFO:tensorflow:Restoring parameters from ./model.ckpt
Weight:
[[-1.3130333  -0.3451763  -1.2388619 ]
 [ 0.14669253 -0.1597623   0.32677302]]
Bias:
[ 0.9568557 -1.3136497  1.5366821]


## Dropout

In [70]:
tf.set_random_seed(123456)
tf.reset_default_graph()

# Input
features = tf.Variable([[0.0, 2.0, 3.0, 4.0], [0.1, 0.2, 0.3, 0.4], [11.0, 12.0, 13.0, 14.0]])

# Weights and biases
hidden_layer_weights = [
    [0.1, 0.2, 0.4],
    [0.4, 0.6, 0.6],
    [0.5, 0.9, 0.1],
    [0.8, 0.2, 0.8]]
out_weights = [
    [0.1, 0.6],
    [0.2, 0.1],
    [0.7, 0.9]]
weights = [
    tf.Variable(hidden_layer_weights),
    tf.Variable(out_weights)]
biases = [
    tf.Variable(tf.zeros(3)),
    tf.Variable(tf.zeros(2))]

# dropout prob
keep_prob = tf.placeholder(tf.float32)

In [71]:
# model
hidden_layer_in = tf.add(tf.matmul(features, weights[0]), biases[0])
hidden_layer_out = tf.nn.relu(hidden_layer_in)
hidden_layer_out = tf.nn.dropout(hidden_layer_out, keep_prob)
logits = tf.add(tf.matmul(hidden_layer_out, weights[1]), biases[1])

In [72]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    output = sess.run(logits, feed_dict={keep_prob: 0.5})
    print(output)

[[ 9.559999  16.       ]
 [ 0.9100001  1.016    ]
 [33.74      43.38     ]]
